In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook



In [2]:
# Load API Credentials
with open('/Users/jezep/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)



In [36]:
LOCATION = 'Chicago, IL'
TERM = 'pizza'


In [37]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_IL_pizza.json"
JSON_FILE



'Data/results_in_progress_IL_pizza.json'

In [38]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_IL_pizza.json not found. Saving empty list to file.


In [39]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')



- 0 previous results found.


In [40]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()



dict_keys(['businesses', 'total', 'region'])

In [41]:
## How many results total?
total_results = results['total']
total_results



5300

In [42]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page



20

In [10]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



590

In [43]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)


In [44]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 


  0%|          | 0/560 [00:00<?, ?it/s]

In [45]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)



  0%|          | 0/560 [00:00<?, ?it/s]

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

In [46]:
## delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)



False

In [47]:
import os

def create_json_file(JSON_FILE, delete_if_exists=False):

    # Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)

    # If it DOES exist:
    if file_exists == True:

        # Check if user wants to delete if exists
        if delete_if_exists==True:

            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            
            # delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            
            # Recursive call to function after old file deleted
            create_json_file(JSON_FILE, delete_if_exists=False)
            
        else:
            print(f"[i] {JSON_FILE} already exists.")       
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  



In [48]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



[i] Data/results_in_progress_IL_pizza.json not found. Saving empty list to new file.
- 0 previous results found.


265

In [49]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)



  0%|          | 0/265 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [51]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(5), final_df.tail(5))


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,DXwSYgiXqIVNdO9dazel6w,pequods-pizzeria-chicago,Pequod's Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/8QJUNb...,False,https://www.yelp.com/biz/pequods-pizzeria-chic...,7688,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 41.92187, 'longitude': -87.664486}","[pickup, delivery]",$$,"{'address1': '2207 N Clybourn Ave', 'address2'...",+17733271512,(773) 327-1512,2158.708458
1,8vFJH_paXsMocmEO_KAa3w,lou-malnatis-pizzeria-chicago,Lou Malnati's Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/abCXuj...,False,https://www.yelp.com/biz/lou-malnatis-pizzeria...,7167,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.890357, 'longitude': -87.633704}","[pickup, delivery]",$$,"{'address1': '439 N Wells St', 'address2': '',...",+13128289800,(312) 828-9800,4000.999053
2,Bxd9Y2PsdvIOGIKFQh53RQ,giordanos-chicago-30,Giordano's,https://s3-media1.fl.yelpcdn.com/bphoto/OKxuR9...,False,https://www.yelp.com/biz/giordanos-chicago-30?...,3584,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.88513231411071, 'longitude': -...",[delivery],$$,"{'address1': '130 E Randolph St', 'address2': ...",+13126161200,(312) 616-1200,4994.255603
3,Nza-kbPzUfFMZGyJz12sYg,the-stopalong-chicago,The StopAlong,https://s3-media2.fl.yelpcdn.com/bphoto/IzWXbV...,False,https://www.yelp.com/biz/the-stopalong-chicago...,319,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 41.91426, 'longitude': -87.68393}","[pickup, delivery]",$$,"{'address1': '1812 N Milwaukee Ave', 'address2...",+17733944694,(773) 394-4694,1136.647564
4,wiNuegM8MNw9WIP0wfMmeA,coalfire-pizza-chicago,Coalfire Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/VYA0aN...,False,https://www.yelp.com/biz/coalfire-pizza-chicag...,1200,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 41.89075, 'longitude': -87.65988}","[delivery, pickup]",$$,"{'address1': '1321 W Grand Ave', 'address2': '...",+13122262625,(312) 226-2625,2170.786489


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,D76HdXFjAqk1Xp05SXdBnA,beggars-pizza-oak-lawn,Beggars Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/xfLdgy...,False,https://www.yelp.com/biz/beggars-pizza-oak-law...,71,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.70601, 'longitude': -87.7603}",[delivery],$$,"{'address1': '10240 Central Ave', 'address2': ...",+17084990505,(708) 499-0505,23176.966594
996,Lo4Pci2so6wUb9ZNp224Mg,navy-pier-beer-garden-chicago,Navy Pier Beer Garden,https://s3-media3.fl.yelpcdn.com/bphoto/OHKNT5...,False,https://www.yelp.com/biz/navy-pier-beer-garden...,69,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",3.5,"{'latitude': 41.8916048731132, 'longitude': -8...",[],$$,"{'address1': '600 E Grand Ave', 'address2': ''...",+13125955439,(312) 595-5439,6545.414008
997,mIa3DWMFUU2KbCACC3OHHg,the-social-table-chicago-2,The Social Table,https://s3-media3.fl.yelpcdn.com/bphoto/JCESXo...,False,https://www.yelp.com/biz/the-social-table-chic...,65,"[{'alias': 'cookingschools', 'title': 'Cooking...",5.0,"{'latitude': 41.9179970776209, 'longitude': -8...",[],$$,"{'address1': '819 W Armitage Ave', 'address2':...",+17736974109,(773) 697-4109,2749.341629
998,j_2R7YinhgSamKA8-wjJkw,museum-campus-cafe-chicago,Museum Campus Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/mG-99W...,False,https://www.yelp.com/biz/museum-campus-cafe-ch...,4,"[{'alias': 'hotdog', 'title': 'Hot Dogs'}, {'a...",2.0,"{'latitude': 41.8674408, 'longitude': -87.6191...",[],$,"{'address1': '1200 S Lake Shore Dr', 'address2...",+17736432077,(773) 643-2077,6788.463648
999,qaY5kcomlU0MT76034trhw,marianos-lakeshore-east-chicago,Mariano's - Lakeshore East,https://s3-media2.fl.yelpcdn.com/bphoto/NpOg_W...,False,https://www.yelp.com/biz/marianos-lakeshore-ea...,453,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.5,"{'latitude': 41.885283, 'longitude': -87.618464}","[pickup, delivery]",$$,"{'address1': '333 E Benton Place', 'address2':...",+13122281349,(312) 228-1349,5382.114252
